In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error
from sklearn import svm

In [3]:
#scale/standardize vars
df = pd.read_csv("archive/data.csv")
df_artist = pd.read_csv("archive/data_by_artist.csv")
df_genres = pd.read_csv("archive/data_by_genres.csv")
df_year = pd.read_csv("archive/data_by_year.csv")
df_w_genres = pd.read_csv("archive/data_w_genres.csv")

In [4]:
X = df[['acousticness', 'danceability','energy','year', 'explicit','instrumentalness', 'key', 'liveness', 'loudness','mode', 
        'speechiness', 'tempo','valence', 'duration_ms']]

In [5]:
y = df['popularity']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
model_linear = LinearRegression()
model_linear.fit(X_train,y_train)
model_linear_acc = model_linear.score(X_test,y_test)
print(model_linear_acc) # highest accuracy of all models
print(model_linear.coef_)  
print(model_linear.rank_) # multicollinearity not an issue between coefs 
# key, loudness, tempo, and duration_ms are very close to 0, so we can remove them from the model

0.7819992419510207
[-4.16896481e+00  4.72605387e+00 -1.17604373e+00  6.84203301e-01
  4.84348050e-01 -2.46761469e+00 -1.19000150e-02 -2.44652417e+00
  3.13309042e-02 -2.53928979e-01 -7.67374515e+00  5.34910801e-03
 -3.68358570e-01  1.61338939e-07]
14


In [8]:
model_linear2 = LinearRegression()
X_train_linear2 = X_train.drop(['key','loudness','tempo','duration_ms'],axis=1)
X_test_linear2 = X_test.drop(['key','loudness','tempo','duration_ms'],axis=1)
model_linear2.fit(X_train_linear2,y_train)
model_linear2_acc = model_linear2.score(X_test_linear2,y_test)
print(model_linear2_acc) # we see that the accuracy is practically unchanged after removing key, loudness, tempo,
                         # and duration_ms
print(model_linear2.coef_) # All coefficients are sufficiently far from 0
print(model_linear2.rank_) # X matrix is still full rank

0.7819860804783374
[-4.13191057  4.65522147 -0.57496169  0.68497233  0.5182307  -2.60205116
 -2.49726699 -0.23773958 -7.79048603 -0.27515854]
10


In [9]:
model_forest = RandomForestClassifier()
model_forest.fit(X_train,y_train)
model_forest_acc = model_forest.score(X_test,y_test)
model_forest_acc

0.19907598140191865

In [10]:
model_forest11 = RandomForestClassifier(max_depth=15,min_samples_split=6,max_features=7)
model_forest11.fit(X_train,y_train)
model_forest11_acc = model_forest11.score(X_test,y_test)
model_forest11_acc # Setting the parameters above results in an improvement in accuracy of about 0.5%

0.20404920251898065

In [15]:
# Extract single tree
estimator = model_forest11.estimators_[0]
print(estimator)
from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', rounded = True, proportion = False, precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=15, max_features=7, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=6,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=509242813, splitter='best')


ModuleNotFoundError: No module named 'anytree'